In [64]:
import pandas as pd
import numpy as np
import scipy.sparse as sparse
import keras
import lightfm
import random
import implicit
import time
from lightfm import cross_validation
from skopt import forest_minimize
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn import metrics
from lightfm.evaluation import precision_at_k,auc_score
from scipy.sparse.linalg import spsolve

In [2]:
data = pd.read_csv('DATA.csv', usecols=[i for i in range(1,19)])
#data.drop_duplicates(inplace=True)

d:\anaconda3\envs\myenv\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (2,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
data.describe(include='all')

,amount,atm_address,atm_address_lat,atm_address_lon,city,country,currency,customer_id,home_add_lat,home_add_lon,mcc,pos_address,pos_adress_lat,pos_adress_lon,terminal_id,transaction_date,work_add_lat,work_add_lon
count,1.046217e+06,217032,148956.000000,148956.000000,1046209,1046217,1.046217e+06,1046217,1.046203e+06,1.046203e+06,1.046217e+06,802573,818130.000000,818130.000000,1046195,1046217,565790.000000,565790.000000
unique,NaN,15764,NaN,NaN,6692,119,NaN,9988,NaN,NaN,NaN,85073,NaN,NaN,196177,247,NaN,NaN
top,NaN,"Череповец, ул. Мира, д. 30",NaN,NaN,MOSCOW,RUS,NaN,5ac9db6c170b412c48870e3f9a60985a,NaN,NaN,NaN,"30,MIRACHEREPOVETS162600 RUSRUS",NaN,NaN,7487298f8a95494300a28b4baa051f85,2017-06-09,NaN,NaN
freq,NaN,1115,NaN,NaN,230284,1037906,NaN,1512,NaN,NaN,NaN,1217,NaN,NaN,506,7045,NaN,NaN
mean,2.816710e+00,NaN,56.075658,40.231044,NaN,NaN,6.440099e+02,NaN,5.613875e+01,4.066272e+01,5.692691e+03,NaN,56.174973,40.340550,NaN,NaN,56.238717,40.446978
std,7.186886e-01,NaN,3.421225,12.808538,NaN,NaN,2.125877e+01,NaN,3.337596e+00,1.357629e+01,5.007829e+02,NaN,3.314418,13.360626,NaN,NaN,3.152634,13.370788
min,-2.012356e+00,NaN,42.053000,20.169000,NaN,NaN,8.000000e+00,NaN,4.211100e+01,1.991000e+01,7.420000e+02,NaN,42.102032,19.887847,NaN,NaN,43.110000,19.911000
25%,2.321968e+00,NaN,55.649000,36.301000,NaN,NaN,6.430000e+02,NaN,5.561800e+01,3.618300e+01,5.411000e+03,NaN,55.645867,36.047079,NaN,NaN,55.702000,36.276000
50%,2.727359e+00,NaN,55.781000,37.602000,NaN,NaN,6.430000e+02,NaN,5.580100e+01,3.759200e+01,5.699000e+03,NaN,55.789657,37.587477,NaN,NaN,55.775000,37.594000
75%,3.256952e+00,NaN,59.122000,37.923000,NaN,NaN,6.430000e+02,NaN,5.911800e+01,3.844000e+01,5.999000e+03,NaN,59.117524,37.982154,NaN,NaN,59.134000,37.819000


In [3]:
print(data.shape)

(1046217, 18)


In [7]:
print(data.iloc[81290])

amount                                                        1.93683
atm_address                                                       NaN
atm_address_lat                                                   NaN
atm_address_lon                                                   NaN
city                                                        KRASNODAR
country                                                           RUS
currency                                                          643
customer_id                          a1e70b29a2ed05f02988bbd93c72747e
home_add_lat                                                   45.104
home_add_lon                                                   39.008
mcc                                                              5411
pos_address         ul.Rostovskoye shosse, d.30KRASNODAR350087    ...
pos_adress_lat                                                 45.098
pos_adress_lon                                                38.9959
terminal_id         

In [8]:
data.head()

,amount,atm_address,atm_address_lat,atm_address_lon,city,country,currency,customer_id,home_add_lat,home_add_lon,mcc,pos_address,pos_adress_lat,pos_adress_lon,terminal_id,transaction_date,work_add_lat,work_add_lon
0,2.884034,NaN,NaN,NaN,ST PETERSBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851,30.232,5261,NaN,59.844072,30.179153,11606fde0c814ce78e0d726e39a0a5ee,2017-07-15,59.847,30.177
1,2.787498,NaN,NaN,NaN,ST PETERSBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851,30.232,5261,NaN,59.844072,30.179153,6c5e5793ebc984fb72875feffff62854,2017-09-09,59.847,30.177
2,2.892510,NaN,NaN,NaN,ST PETERSBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851,30.232,5261,NaN,59.844072,30.179153,0576445d74e374c92c0902e612fca356,2017-07-06,59.847,30.177
3,2.909018,NaN,NaN,NaN,ST PETERSBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851,30.232,5261,NaN,59.844072,30.179153,af0d099257ccedbfe730c63a491e0d84,2017-08-23,59.847,30.177
4,2.801228,NaN,NaN,NaN,ST PETERSBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851,30.232,5261,NaN,59.844072,30.179153,0576445d74e374c92c0902e612fca356,2017-07-29,59.847,30.177


In [9]:
data.tail()

,amount,atm_address,atm_address_lat,atm_address_lon,city,country,currency,customer_id,home_add_lat,home_add_lon,mcc,pos_address,pos_adress_lat,pos_adress_lon,terminal_id,transaction_date,work_add_lat,work_add_lon
1047185,2.601024,NaN,NaN,NaN,MOSKVA,RUS,643.0,381bcceaad6c0d72a795e2f09950b3af,55.676,37.872,6011,NaN,NaN,NaN,fff0535ce27b0551cb1da1ef6852895e,2017-03-06,NaN,NaN
1047186,2.717837,NaN,NaN,NaN,MOSKVA,RUS,643.0,90ffc453a1c176755ef9225a77df81a1,55.502,37.582,6011,NaN,NaN,NaN,fff0535ce27b0551cb1da1ef6852895e,2017-03-18,NaN,NaN
1047187,2.859485,NaN,NaN,NaN,MOSKVA,RUS,643.0,90ffc453a1c176755ef9225a77df81a1,55.502,37.582,6011,NaN,NaN,NaN,fff0535ce27b0551cb1da1ef6852895e,2017-02-26,NaN,NaN
1047188,3.715220,NaN,NaN,NaN,MOSKVA,RUS,643.0,5b4e3e9f63d011a61a8d4fa065b6fa97,55.716,37.787,6011,NaN,NaN,NaN,fff0535ce27b0551cb1da1ef6852895e,2017-02-20,55.734,37.662
1047189,3.865356,NaN,NaN,NaN,MOSKVA,RU,643.0,a5dddded8ca6fee2441dd4f930e38d0d,55.652,37.536,6011,NaN,NaN,NaN,fff12255d4b209aef890e66ddb7951b0,2017-02-24,55.752,37.535


In [10]:
print(len(data[data.customer_id == '0dc0137d280a2a82d2dc89282450ff1b']))
print(len(data[data.customer_id == '381bcceaad6c0d72a795e2f09950b3af']))
print(len(data[data.customer_id == '90ffc453a1c176755ef9225a77df81a1']))
print(len(data[data.customer_id == '5b4e3e9f63d011a61a8d4fa065b6fa97']))
print(len(data[data.customer_id == 'a5dddded8ca6fee2441dd4f930e38d0d']))

50
81
119
203
226


# preproc

In [11]:
def del_col(data, num):
    for n in num:
        if n not in data.columns:
            return data    
    return data.drop(num, axis=1)

def lower_strip(data):
    data['city'] = data.city.str.lower()
    data['city'] = data.city.str.strip()
    data['country'] = data.country.str.lower()
    data['country'] = data.country.str.strip()
    return data
    
def change_cntr(data, cntry_a, cntry_b):
    data.loc[data.country == cntry_a, 'country'] = cntry_b
    return data

def change_city(data, city_a, city_b):
    data.loc[data.city == city_a, 'city'] = city_b
    return data

def fillna_city(data):
    data.loc[data['city'].isnull(),'city']=' '
    return data

def fillna_terminal(data):
    data.loc[data['terminal_id'].isnull(), 'terminal_id']='0'
    return data

def convert_month(data):
    data['transaction_date'] = data['transaction_date'].map(lambda x: str(x)[5:7])
    return data

def encode_customers(data):
    customers = set(data['customer_id'])
    return customers

def sorting(data):
    data = data.sort_values(['customer_id', 'transaction_date'])
    return data

def encode_city_cust_cntr_term_date(data):
    data['city'] = LabelEncoder().fit_transform(data.city)
    data['city'] = data['city'].astype('category')
    data['country'] = LabelEncoder().fit_transform(data.country)
    data['country'] = data['country'].astype('category')
    data['terminal_id'] = LabelEncoder().fit_transform(data.terminal_id)
    data['terminal_id'] = data['terminal_id'].astype('category')
    data['customer_id'] = LabelEncoder().fit_transform(data.customer_id)
    data['customer_id'] = data['customer_id'].astype('category')
    data['transaction_date'] = LabelEncoder().fit_transform(data.transaction_date)
    data['transaction_date'] = data['transaction_date'].astype('category')
    return data

In [12]:
data = del_col(data,['atm_address','atm_address_lat','atm_address_lon',
                     'pos_address','work_add_lat','work_add_lon', 'pos_adress_lat', 'pos_adress_lon',
                     'home_add_lat','home_add_lon'])
data = lower_strip(data)
data = fillna_city(data)
customers = encode_customers(data)
data = sorting(data)

In [13]:
data = convert_month(data)

In [14]:
print(sorted(set(data['city'])))
print(sorted(set(data['country'])))

[' ', '%labuan 2*', '******', '************', '******-**-***', ',', '-', '-- ne zadan -', '.', '.      ---', '0', '010  mar', '0china0000000', '105, ivana go', '108 km minsko', '1119 km "don"', '11190 rec 11', '117209', '11820 filial', '12 km baykals', '127562', '1276 km a-d "', '136,5 km avto', '1375+300 km a', '154 km a/d vo', '155 km a/d m-', '155 km ad mos', '165 km a-d mo', '1660 a/d m-33', '173 89651', '21 km kaluzhs', '21 km murmans', '23, 1, opolch', '24+500 km a-d', '306+300 km a-', '311 km a/d vo', '33, 3, isakov', '33-y km yaros', '416 km a/d mo', '45268', '53 km', '547020000200', '588020000200', '611022000200', '614022000200', '619020000200', '635022000200', '664022000200', '69125 lyon', '77 - moscow', '833022000200', '87 km mkad', 'a          g', 'aba     >', 'abakan', 'aban', 'abatskoe', 'abdulino', 'abdulino g', 'aberdeen', 'abinsk', 'abinsk region', 'abramcevo', 'abramtsevo', 'abrau dyurso', 'abrau-dyurso', 'abu dhabi', 'achinsk', 'adigeya', 'adir', 'adle', 'adler', 'ad

In [15]:
print(sorted(set(data['transaction_date'])))
print(sorted(set(data['currency'])))
print(len(data[data.currency == 643]))
print(len(data[data.city == 'moscow']))

['01', '02', '03', '04', '05', '06', '07', '08', '09']
[8.0, 36.0, 51.0, 124.0, 144.0, 156.0, 191.0, 203.0, 208.0, 214.0, 344.0, 348.0, 352.0, 356.0, 360.0, 376.0, 388.0, 392.0, 398.0, 400.0, 404.0, 417.0, 458.0, 484.0, 498.0, 504.0, 554.0, 578.0, 643.0, 682.0, 702.0, 704.0, 710.0, 752.0, 756.0, 764.0, 784.0, 788.0, 807.0, 818.0, 826.0, 834.0, 840.0, 933.0, 941.0, 944.0, 946.0, 949.0, 972.0, 975.0, 977.0, 978.0, 980.0, 981.0, 985.0]
1041768
243634


In [16]:
cntrs = [['az','aze'],['at','aut'],['bg','bgr'],['ch','che'],['chn','cn'],['ru','rus'],
         ['th','tha'],['ua','ukr'],['de','deu'],['fi','fin'],['fr','fra'],['gb','gbr'],
         ['ge','geo'],['gr','grc'],['hk','hkg'],['hr','hrv'],['hu','hun'],['id','idn'],
         ['in','ind'],['it','ita'],['lt','ltu'],['lv','lva'],['md','mda'],['my','mys'],
         ['nl','nld'],['pl','pol'],['pt','prt'],['us','usa'],['uz','uzb'],['au','aut'],
         ['kg','kgz'],['cz','cze'],['tr','tur']]

cities = {'zell':['zell/see'],'zasechnoe':['zasechnoe s'],'tomsk':['tomsk g'],
          'tolyatti':['tolyatti g',"tol'yatti",'tolyat','toleyatti','toliatti'],
          'leningr_obl':['leningr obl', 'leningr. obl.', 'leningrad ob', 'leningrad obl', 'leningrad reg', 
                         'leningrad.obl', 'leningradsk o', 'leningradsk.o', 'leningradska', 'leningradskay', 
                         'leningradskii'],
          'st_petersburg':['st.-peterburg', 'st.-petersbur','st.peresburg', 'st.peterburg', 'st petersburg'
                           'st.petresburg', 'st.petrsburg', 'st/petersburg', 'st.petersburg'
                           'st-peter', 'st-peter.', 'st-peterb.', 'st-peterbur', 'st-peterburg', 
                           'st-petersburg', 'st-peterurg', 'st. peterburg', 'st. petersbur','s.-peterburg',
                           'sank-peterbur', 'sankt - peter', 'sankt pererbu', 'sankt peterbu', 
                           'sankt petersb', 'sankt-pererbu', 'sankt-petebur', 'sankt-peter', 
                           'sankt-peterbo', 'sankt-peterbu', 'sankt-peters', 'sankt-petersb', 
                           'sankt-peterub', 'sankt-prtrrbu', 'sankt-pterbur', 'sanqt peterbu', 'sant peterbur'],
          'rostovnadonu':['rostov-n', 'rostov-na -do', 'rostov-na-do', 'rostov-na-don', 'rostov-on-don', 'rostov.na.don', 
                          'rostov/donu'],
          'nizh_novgorod':['nizhniy no', 'nizhniy novg', 'nizhniy novgo','nizhniyj novg','nizniy novgor', 'niznov',
              'nizhhiy novgo', 'nizhn.novgord','nizgniy novgo', 'nizh novgorod'],
          'mytishi':['myticshi', 'mytischi', 'mytischi mos', 'mytischy', 'mytish', 'mytishchi', 'mytishchi g', 
                    'mytishchinski', 'mytishchy', 'mytishci', 'mytishhi', 'mytishi', 'mytishy', 'mytyshci',
                    'miticshi', 'mitischi', 'mitishchi', 'mitishchi,', 'mitishci', 'mitishi', 'mityshchy', 'miytischi'],
          'moskva':['moscow', 'moskv*', 'moskva g', 'moskva g sche', 'moscva','mosva',
                    'moskva g. tro', 'moskva g. vos', 'moskva g. zel', 'moskva g.kras', 'moskva g.mosk', 
                    'moskva g.shch', 'moskva g.zele', 'moskva gsp-1', 'moskva rublev', 'moskva zeleno', 
                    'moskva,', 'moskva, cev.', 'moskva, g. mo', 'moskva, g. tr', 'moskva, g. ze', 
                    'moskva, g.kra', 'moskva, g.mos', 'moskva, g.zel', 'moskva, gor.', 'moskva, kommu', 'moskva, moskv',  
                    'moskva, scher', 'moskva, sever', 'moskva, zelen', 'moskva, zhr k', 'moskva,tits s', 
                    'moskva,zeleno', 'moskva-zeleno', 'moskva.', 'moskvaa', 'moskve', 'moskvy', 'moskwa', 'mosocw'],
          'moskva_obl':['moscow  regio', 'moscow obl.', 'moscow oblast', 'moscow reg', 'moskow region','moskovsk.obl.',
                        'moscow reg.', 'moscow region', 'moscow, pos.', 'moscow, posel','moskovsk obl','moskovkaya ob',
                        'moskovskaya o','moskowskaya o','moskva region','moskva p.vnuk', 'moskva pos. d', 'moskva pos. m', 
                        'moskva pos. s', 'moskva pos. v', 'moskva p mosk', 'moskva p sose', 'moskva p. mos', 'moskva, pos.'
                        'moskva p. per', 'moskva p. sos', 'moskva p. vnu', 'moskva p.koko', 'moskva p.mosk', 'moskva p.sose',
                        'moskva,der.fo', 'moskva,p. kom', 'moskva,p. vor', 'moskva,p.maru', 'mosobl', 'moskva, p. de', 
                        'moskva, p. pt', 'moskva, p. so', 'moskva, p. vn', 'moskva,p.perv', 'moskva,p.sose', 'moscowskaya o',
                        'moskva,posele', 'moskva, p.rog', 'moskva, p.sos', 'moskva, p.vnu','moskva obl','mo',
                        'moskva, pos.', 'moskva,p. vnu'],
          'novorossiysk':['novoroosiiysk', 'novorosiysk', 'novorossiisk', 'novorossiiysk', 'novorossijsk', 
              'novorossisk', 'novorossiysk,', 'novorossiysko', 'novorossyisk', 'novorossysk'],
          'dolgoprudniy':['dolgoprudnii', 'dolgoprudnnyy', 'dolgoprudny', 'dolgoprudnyi', 
              'dolgoprudnyiy', 'dolgoprudnyj', 'dolgoprudnyy', 'dolgoprudnyy,'] # and else ..
         }
for c in cities:
    for j in range(len(cities[c])):
        data = change_city(data, cities[c][j], c)
for i in range(len(cntrs)):
    data = change_cntr(data, cntrs[i][0], cntrs[i][1])

In [18]:
data = fillna_terminal(data)
data = encode_city_cust_cntr_term_date(data)

In [78]:
data['mcc'] = data['mcc'].astype('category')
data.describe(include='all')

,amount,city,country,currency,customer_id,mcc,terminal_id,transaction_date
count,1.047190e+06,1047190.0,1047190.0,1.047190e+06,1047190.0,1047190.0,1047190.0,1047190.0
unique,NaN,5186.0,86.0,NaN,9988.0,243.0,196178.0,9.0
top,NaN,2584.0,68.0,NaN,9607.0,5411.0,89223.0,5.0
freq,NaN,392466.0,1044054.0,NaN,1643.0,340858.0,506.0,174555.0
mean,2.816540e+00,NaN,NaN,6.440090e+02,NaN,NaN,NaN,NaN
std,7.185389e-01,NaN,NaN,2.124891e+01,NaN,NaN,NaN,NaN
min,-2.012356e+00,NaN,NaN,8.000000e+00,NaN,NaN,NaN,NaN
25%,2.321954e+00,NaN,NaN,6.430000e+02,NaN,NaN,NaN,NaN
50%,2.727206e+00,NaN,NaN,6.430000e+02,NaN,NaN,NaN,NaN
75%,3.256577e+00,NaN,NaN,6.430000e+02,NaN,NaN,NaN,NaN


In [20]:
data.dtypes

amount               float64
city                category
country             category
currency             float64
customer_id         category
mcc                    int64
terminal_id         category
transaction_date    category
dtype: object

In [122]:
#d2 = data['customer_id', 'mcc', 'transaction_date'].copy()
d2 = pd.DataFrame()
print(len(data['mcc'].to_numpy()))

1047190


In [148]:
print(data['mcc'].cat.categories)

Int64Index([ 742,  763,  780, 1711, 1731, 1750, 1761, 1799, 2741, 2842,
            ...
            8398, 8661, 8699, 8911, 8999, 9211, 9222, 9311, 9399, 9402],
           dtype='int64', length=243)


In [ ]:
buskets = pd.DataFrame()
for u in range(:9988):
    buskets.append()

# computing

#### rnn 

In [3]:
"""def make_buskets(data):
    buskets = {i:{j:[data.loc[(data['customer_id']==i) & (data['transaction_date']==j),'mcc'].to_numpy()]
               for j in range(9)} for i in range(9988)}
    return buskets"""

def custom_loss(y, y_pred):
    loss = 0
    """     
    for u,i,j in loss_samples:
        x = predict(u,i) - predict(u,j)
        ranking_loss += 1.0/(1.0+exp(x))"""
    return loss  

def get_batch(X_, y_=None, batch_size=-1):
    n_samples = X_.shape[0]

    if batch_size == -1:
        batch_size = n_samples

    for i in range(0, n_samples, batch_size):
        upper_bound = min(i + batch_size, n_samples)
        ret_x = X_[i:upper_bound]
        ret_y = None
        if y_ is not None:
            ret_y = y_[i:i + batch_size]
            yield (ret_x, ret_y)

def rate_column(data):
    r = []
    keys = namedtuple("keys", "customer_id mcc".split())
    seen = {}
    for i in range(len(data['customer_id'])):
        cust = data.loc[i,'customer_id']
        mcc_ = data.loc[i, 'mcc']
        if (cust, mcc_) in seen.keys():
            r.append(seen[(cust,mcc_)])
        else:
            count = data[(data.customer_id == cust)&(data.mcc==mcc_)].shape[0]
            seen = {keys(customer_id=cust, mcc=mcc_): count}
            r.append(count)
        print(i)
    data['rate']  = r
    return data

In [1]:
#buskets = make_buskets(data)

In [ ]:
Steps = 9
Features = 243

inp = keras.Input(shape=(Steps, Features))
x = keras.layers.Embedding(Features,50)(inp)
x = keras.layers.MaxPooling2D((1,Features))(x)
out = keras.layers.SimpleRNN(Features, activation='sigmoid', kernel_regularizer='l2', 
                             recurrent_regularizer='l2', activity_regularizer='l2',
                             use_bias = False)(x)
#x = keras.Dense(243, activation='sigmoid')(x)

model = keras.Model(inputs=inp, outputs=out)
model.compile(loss=custom_loss, optimizer='adam')

In [ ]:
U = data.customer_id.unique()
pr = []
for i in U:
    model.fit(B, y_train, epochs=15)
    pr.append(model.predict(B))

#### neuMF

In [43]:
data = pd.read_csv('DATA.csv', usecols=[8,11])

In [62]:
def make_train(data,user_col,item_col,rating_col):
    data['rate']=[1 for i in range(data.shape[0])]
    interactions = data.groupby([user_col, item_col])[rating_col] \
            .min().unstack().reset_index().fillna(0).set_index(user_col)
    train = sparse.dok_matrix(interactions, dtype=np.float32)
    return train
def get_train_instances(train, num_negatives):
    user_input, item_input, labels = [],[],[]
    num_users, num_items = train.shape
    #print(num_items)
    for (u, i) in train.keys():
        # positive instance
        user_input.append(u)
        item_input.append(i)
        labels.append(1)
        # negative instances
        for t in range(num_negatives):
            j = np.random.randint(num_items)
            while (u, j) in train.keys():
                j = np.random.randint(num_items)
            user_input.append(u)
            item_input.append(j)
            labels.append(0)
    return user_input, item_input, labels

In [45]:
train = make_train(data,'customer_id','mcc','rate')

In [46]:
n_latent_factors_user = 8
n_latent_factors_item = 10
n_latent_factors_mf = 3
n_users, n_items = len(data.customer_id.unique()), len(data.mcc.unique())

In [58]:
item_input = keras.layers.Input(shape=[1],name='Item')
item_embedding_mlp = keras.layers.Embedding(n_items + 1, n_latent_factors_item, name='Item-Embedding-MLP')(item_input)
item_vec_mlp = keras.layers.Flatten(name='FlattenItems-MLP')(item_embedding_mlp)
item_vec_mlp = keras.layers.Dropout(0.2)(item_vec_mlp)

item_embedding_mf = keras.layers.Embedding(n_items + 1, n_latent_factors_mf, name='Item-Embedding-MF')(item_input)
item_vec_mf = keras.layers.Flatten(name='FlattenItems-MF')(item_embedding_mf)
item_vec_mf = keras.layers.Dropout(0.2)(item_vec_mf)

user_input = keras.layers.Input(shape=[1],name='User')
user_vec_mlp = keras.layers.Flatten(name='FlattenUsers-MLP')(keras.layers.Embedding(n_users + 1, n_latent_factors_user,name='User-Embedding-MLP')(user_input))
user_vec_mlp = keras.layers.Dropout(0.2)(user_vec_mlp)

user_vec_mf = keras.layers.Flatten(name='FlattenUsers-MF')(keras.layers.Embedding(n_users + 1, n_latent_factors_mf,name='User-Embedding-MF')(user_input))
user_vec_mf = keras.layers.Dropout(0.2)(user_vec_mf)

concat = keras.layers.concatenate([item_vec_mlp, user_vec_mlp],name='Concat')
concat_dropout = keras.layers.Dropout(0.2)(concat)
dense = keras.layers.Dense(200,name='FullyConnected')(concat_dropout)
dense_batch = keras.layers.BatchNormalization(name='Batch')(dense)
dropout_1 = keras.layers.Dropout(0.2,name='Dropout-1')(dense_batch)
dense_2 = keras.layers.Dense(100,name='FullyConnected-1')(dropout_1)
dense_batch_2 = keras.layers.BatchNormalization(name='Batch-2')(dense_2)

dropout_2 = keras.layers.Dropout(0.2,name='Dropout-2')(dense_batch_2)
dense_3 = keras.layers.Dense(50,name='FullyConnected-2')(dropout_2)
dense_4 = keras.layers.Dense(20,name='FullyConnected-3', activation='relu')(dense_3)

pred_mf = keras.layers.dot([item_vec_mf, user_vec_mf],axes=-1,name='Dot')

pred_mlp = keras.layers.Dense(1, activation='relu',name='Activation')(dense_4)

combine_mlp_mf = keras.layers.concatenate([pred_mf, pred_mlp],name='Concat-MF-MLP')
result_combine = keras.layers.Dense(100,name='Combine-MF-MLP')(combine_mlp_mf)
deep_combine = keras.layers.Dense(100,name='FullyConnected-4')(result_combine)

result = keras.layers.Dense(1,name='Prediction')(deep_combine)

model = keras.Model([user_input, item_input], result)
opt = keras.optimizers.Adam(lr =0.01)
model.compile(optimizer='adam',loss= 'mean_absolute_error')

(?, ?) Tensor("dropout_50/cond/Merge:0", shape=(?, ?), dtype=float32)


In [69]:
num_epochs = 5
for ep in range(25):
    t1 = time.time()
    user_input, item_input, labels = get_train_instances(train, num_negatives=4)
    
    hist = model.fit([np.array(user_input), np.array(item_input)], 
                        np.array(labels), 
                        batch_size=256, epochs=1, verbose=0, shuffle=True)
    t2 = time.time()
    print('Iteration %d [%.1f s]: loss = %.4f'% (ep, t2-t1, hist.history['loss'][0]))

Iteration 0 [13.6 s]: loss = 0.1046
Iteration 1 [14.4 s]: loss = 0.1045
Iteration 2 [13.6 s]: loss = 0.1045
Iteration 3 [13.8 s]: loss = 0.1041
Iteration 4 [13.7 s]: loss = 0.1042
Iteration 5 [13.8 s]: loss = 0.1040
Iteration 6 [13.7 s]: loss = 0.1038
Iteration 7 [13.9 s]: loss = 0.1040
Iteration 8 [13.7 s]: loss = 0.1034
Iteration 9 [13.7 s]: loss = 0.1042
Iteration 10 [13.7 s]: loss = 0.1034
Iteration 11 [13.7 s]: loss = 0.1034
Iteration 12 [13.8 s]: loss = 0.1027
Iteration 13 [13.8 s]: loss = 0.1018
Iteration 14 [13.9 s]: loss = 0.1012
Iteration 15 [13.9 s]: loss = 0.1005
Iteration 16 [14.2 s]: loss = 0.1003
Iteration 17 [13.9 s]: loss = 0.0998
Iteration 18 [13.9 s]: loss = 0.0993
Iteration 19 [14.0 s]: loss = 0.0996
Iteration 20 [13.9 s]: loss = 0.0992
Iteration 21 [13.8 s]: loss = 0.0995
Iteration 22 [13.9 s]: loss = 0.0992
Iteration 23 [14.1 s]: loss = 0.0988
Iteration 24 [14.7 s]: loss = 0.0986


In [110]:
print(data.mcc.unique()[133])

7911


In [115]:
submission_NeuMF = pd.DataFrame()
i = 0
for c in sorted(data.customer_id.unique()):
    us = np.full(243, i, dtype = 'int32')
    scores = np.array(model.predict([us,np.arange(243)],batch_size=243, verbose=0))
    scores = scores.ravel()
    indexes = np.argpartition(scores, -5)[-5:]
    pred = le.inverse_transform(indexes)
    submission_NeuMF[c] = pred
    i+=1

In [116]:
submission_NeuMF

,0001f322716470bf9bfc1708f06f00fc,0007297d86e14bd68bd87b1dbdefe302,000b709c6c6fb1e8efcfd95e57c2a9de,0027a7618d97cc9fbda55fac457eaeb7,002b9f6e118c54f1292e03d1a04d516e,0031915eb230f772681fb5dc5a8d1c31,003360bff9882ca4a4f93394dd984822,0037f3de3d890df1022cc760a1dfd9d6,003fa58414cc55531fcc38423bea8f8e,0046c2952fb808aa11f74abce5abe097,...,ff70c3c5f2dca00f255705305d75111d,ff71bdbcba59047f1fad88dcb7052151,ff7a887d347a8d598dc8e559d3aaec2f,ff7e1c6c07469b28a07847540385e767,ff869ee855dc3f9b382c943eb43cc4ec,ff92d5420f5fb92a37e1280d1fc9e5f4,ffaeae55d4dbf29058f04e7a6a764f02,ffc5289194413ec68c3f7adc8121d69b,ffd097949a4a238296a7deadfb376cc0,ffdd5ec2a90e355cf40525eac1a6fd34
0,5814,5661,5812,5541,5541,5912,5912,5912,5691,5541,...,5651,6011,5814,5661,5541,5651,5541,5812,5651,5912
1,5912,5912,5651,5651,5912,5921,5691,5814,5912,5411,...,5661,5912,5331,5912,5499,5499,5499,5411,5812,5541
2,5921,5411,5691,5691,5812,5814,5311,5691,5411,5651,...,5812,5411,5499,5812,5641,8099,5814,5331,5411,5812
3,5411,5331,5912,5912,5331,5311,5411,5411,5812,5912,...,5331,5331,5641,5331,8099,5311,5912,5814,5912,5651
4,5691,5812,5411,5411,5411,5641,5651,5921,5541,5812,...,5499,5814,8099,5411,5311,5641,5411,5912,5541,5411


#### MF with warp loss

In [5]:
data = pd.read_csv('DATA.csv', usecols=[8,11])

In [31]:
def runMF(interactions, n_components=30,loss='warp',epoch=20,learning_rate=0.1, 
          learning_schedule='adagrad',alpha=0.1,k=5):
    model = lightfm.LightFM(loss=loss,
                            random_state=42,
                            learning_schedule=learning_schedule,
                            learning_rate=learning_rate,
                            no_components= n_components,
                            user_alpha = alpha,
                            item_alpha = alpha,
                            k=k)
    model.fit(interactions,epochs=epoch)
    return model
def sample_recommendation_user(model,matrix,customer_id,mcc_dict,user_dict,topk=5):
    n_users, n_items = matrix.shape
    scores = np.array(mf_model.predict(user_dict[customer_id],np.arange(n_items)))
    indexes = np.argpartition(scores, -topk)[-topk:]
    pred = []
    for i in indexes:
        pred.append(mcc_dict[i])
    return pred
def user_dict(customers):
    user_d = {}
    for i,id_ in enumerate(customers):
        user_d[id_] = i
    return user_d
def dop(cust):
    d = {}
    for i in data.loc[(data.customer_id==cust),'mcc']:
        if i not in d.keys():
            d[i]=1
        else:
            d[i]+=1
    return d
def dec(customers):
    decode = {}
    for i,c in enumerate(customers):
        uniq = {}
        d = dop(c)
        ind = list(UI_sparse[i].indices)
        data = list(UI_sparse[i].data)
        for j in range(len(data)):
            if data[j] not in uniq.keys():
                uniq[data[j]] = ind[j]
        for j in d:
            if d[j] in uniq.keys():
                if uniq[d[j]] not in decode.keys():
                    decode[uniq[d[j]]] = j
    return decode
def objective(params):
    loss,epochs,learning_rate,no_components, alpha = params
    
    user_alpha = alpha
    item_alpha = alpha
    model = lightfm.LightFM(loss='warp',
                    random_state=42,
                    learning_rate=learning_rate,
                    no_components=no_components,
                    user_alpha=user_alpha,
                    item_alpha=item_alpha)
    model.fit(train, epochs=epochs,verbose=True)
    
    patks = lightfm.evaluation.precision_at_k(model, valid,
                                              train_interactions=None,
                                              k=5)
    mapatk = np.mean(patks)
    out = -mapatk
    if np.abs(out + 1) < 0.01 or out < -1.0:
        return 0.0
    else:
        return out

In [4]:
data['rate'] = [1 for i in range(data.shape[0])]

In [160]:
customers = list(np.sort(data.customer_id.unique()))
mccs = list(data.mcc.unique()) 
rates = list(data.rate) 

rows = data.customer_id.astype('category').cat.codes 
cols = data.mcc.astype('category').cat.codes 
UI_sparse = sparse.csr_matrix((rates, (rows, cols)), shape=(len(customers), len(mccs)))

In [6]:
train, valid = cross_validation.random_train_test_split(UI_sparse, test_percentage=0.2)

In [19]:
space = [('warp','warp-cos'), #loss
         (1, 100), # epochs
         (0.1,0.01,0.001), # learning_rate
         (5, 20), # no_components
         (10**-6, 10**-1, 'log-uniform'), # alpha
        ]

res_fm = forest_minimize(objective, space, n_calls=250,
                     random_state=0,
                     verbose=True)

Iteration No: 1 started. Evaluating function at random point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 21.8745
Function value obtained: -0.1741
Current minimum: -0.1741
Iteration No: 2 started. Evaluating function at random point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 

Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Iteration No: 34 ended. Search finished for the next optimal point.
Time taken: 23.5794
Function value obtained: -0.1741
Current minimum: -0.1744
Iteration No: 35 started. Searching for the next optimal point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11

Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Iteration No: 64 ended. Search finished for the next optimal point.
Time taken: 20.0618
Function value obtained: -0.1100
Current minimum: -0.1756
Iteration No: 65 started. Searching for the next optimal point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Iteration No: 65 ended. Search finished for the next optimal point.
Time taken: 7.4162
Function value obtained: -0.1741
Current minimum: -0.1756
Iteration No: 66 started. Searching for the next optimal point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 2

Function value obtained: -0.1740
Current minimum: -0.1756
Iteration No: 96 started. Searching for the next optimal point.
Epoch 0
Iteration No: 96 ended. Search finished for the next optimal point.
Time taken: 0.9397
Function value obtained: -0.1569
Current minimum: -0.1756
Iteration No: 97 started. Searching for the next optimal point.
Epoch 0
Iteration No: 97 ended. Search finished for the next optimal point.
Time taken: 0.7685
Function value obtained: -0.1741
Current minimum: -0.1756
Iteration No: 98 started. Searching for the next optimal point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Iteration No: 98 ended. Search finished for the next optimal point.
Time taken: 4.1813
Function value obtained: -0.1702
Current minimum: -0.1756
Iteration No: 99 started. Searching for the next optimal point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Iteration No: 99 ended. Search finished for the next optimal point.
Time taken: 3.4201
Function value obtained: -0.1239
Cur

Iteration No: 159 started. Searching for the next optimal point.
Epoch 0
Iteration No: 159 ended. Search finished for the next optimal point.
Time taken: 0.7691
Function value obtained: -0.1533
Current minimum: -0.1756
Iteration No: 160 started. Searching for the next optimal point.
Epoch 0
Iteration No: 160 ended. Search finished for the next optimal point.
Time taken: 0.6822
Function value obtained: -0.0100
Current minimum: -0.1756
Iteration No: 161 started. Searching for the next optimal point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Iteration No: 161 ended. Search finished for the next optimal point.
Time taken: 6.3527
Function value obtained: -0.1741
Current minimum: -0.1756
Iteration No: 162 started. Searching for the next optimal point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8

Iteration No: 227 ended. Search finished for the next optimal point.
Time taken: 0.5966
Function value obtained: -0.1741
Current minimum: -0.1756
Iteration No: 228 started. Searching for the next optimal point.
Epoch 0
Iteration No: 228 ended. Search finished for the next optimal point.
Time taken: 0.6709
Function value obtained: -0.1535
Current minimum: -0.1756
Iteration No: 229 started. Searching for the next optimal point.
Epoch 0
Epoch 1
Iteration No: 229 ended. Search finished for the next optimal point.
Time taken: 0.9315
Function value obtained: -0.1744
Current minimum: -0.1756
Iteration No: 230 started. Searching for the next optimal point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Iteration No: 230 ended. Search finished for the next optimal point.
Time taken: 3.1071
Function value obtained: -0.1741
Current minimum: -0.1756
Iteration No: 231 started. Searching for the next optimal point.
Epoch 0
Iteration No: 231

In [21]:
print('Maximimum p@k found: {:6.5f}'.format(-res_fm.fun))
print('Optimal parameters:')
params = ['loss', 'epochs', 'learning_rate', 'no_components', 'alpha']
for (p, x_) in zip(params, res_fm.x):
    print('{}: {}'.format(p, x_))

Maximimum p@k found: 0.17563
Optimal parameters:
loss: warp-cos
epochs: 41
learning_rate: 0.1
no_components: 16
alpha: 0.0017850764991343868


In [129]:
mf_model = runMF(interactions = UI_sparse,
                 n_components = 16,
                 loss = 'warp-kos',
                 epoch = 41,
                 learning_rate=0.1,
                 learning_schedule='adagrad',
                 alpha = 0.0017850764991343868,
                 k=5)

In [161]:
mcc_dict = dec(mccs)

In [291]:
user_d = user_dict(customers)

In [280]:
submission_MF_warp = pd.DataFrame()
for c in customers:
    submission_MF_warp[c] = sample_recommendation_user(model = mf_model, 
                                      matrix = UI_sparse, 
                                      customer_id = c,
                                      mcc_dict = mcc_dict,
                                      user_dict=user_d)

In [292]:
submission_MF_warp

,0001f322716470bf9bfc1708f06f00fc,0007297d86e14bd68bd87b1dbdefe302,000b709c6c6fb1e8efcfd95e57c2a9de,0027a7618d97cc9fbda55fac457eaeb7,002b9f6e118c54f1292e03d1a04d516e,0031915eb230f772681fb5dc5a8d1c31,003360bff9882ca4a4f93394dd984822,0037f3de3d890df1022cc760a1dfd9d6,003fa58414cc55531fcc38423bea8f8e,0046c2952fb808aa11f74abce5abe097,...,ff70c3c5f2dca00f255705305d75111d,ff71bdbcba59047f1fad88dcb7052151,ff7a887d347a8d598dc8e559d3aaec2f,ff7e1c6c07469b28a07847540385e767,ff869ee855dc3f9b382c943eb43cc4ec,ff92d5420f5fb92a37e1280d1fc9e5f4,ffaeae55d4dbf29058f04e7a6a764f02,ffc5289194413ec68c3f7adc8121d69b,ffd097949a4a238296a7deadfb376cc0,ffdd5ec2a90e355cf40525eac1a6fd34
0,5499,5912,4111,5912,5691,5812,5211,5499,5812,5691,...,4111,5912,5541,5912,5912,5814,5912,5912,5912,5691
1,5541,6011,5814,5814,5912,5814,5311,5814,5912,5912,...,5912,7230,5912,5814,5499,5541,8099,5411,6011,6011
2,5211,5812,6011,5411,5814,5912,5912,5912,6011,5814,...,6011,5499,5814,6011,5921,5499,5411,5812,5814,5411
3,6011,5411,5411,6011,6011,6011,6011,5411,5411,6011,...,5814,6011,5411,8099,6011,6011,6011,6011,5912,5812
4,5411,5814,5912,4111,5411,5411,5411,6011,5814,5411,...,5411,5411,6011,5411,5411,5411,5691,5814,5411,5814


#### als MF

In [6]:
def make_split(ratings, validation_percent = 0.2):
    test_set = ratings.copy() 
    test_set[test_set != 0] = 1 
    training_set = ratings.copy() 
    nonzero_inds = training_set.nonzero() 
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1]))
    random.seed(0)
    num_samples = int(np.ceil(validation_percent*len(nonzero_pairs)))
    samples = random.sample(nonzero_pairs, num_samples) 
    user_inds = [index[0] for index in samples] 
    item_inds = [index[1] for index in samples] 
    training_set[user_inds, item_inds] = 0 
    training_set.eliminate_zeros() 
    return training_set, test_set, list(set(user_inds))
def auc_score(predictions, test):
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    store_auc = [] 
    popularity_auc = [] 
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1)
    item_vecs = predictions[1]
    for user in altered_users: 
        training_row = training_set[user,:].toarray().reshape(-1)
        zero_inds = np.where(training_row == 0) 
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        pop = pop_items[zero_inds] 
        store_auc.append(auc_score(pred, actual)) 
        popularity_auc.append(auc_score(pop, actual)) 
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))
def rec_items(customer_id, data, user_vecs, item_vecs, customer_list, item_list, num_items=5):
    #print(user_vecs, item_vecs)
    cust_ind = np.where(customer_list == customer_id)[0][0] 
    pref_vec = data[cust_ind,:].toarray() 
    pref_vec = pref_vec.reshape(-1) + 1
    pref_vec[pref_vec > 1] = 0
    rec_vector = user_vecs[cust_ind,:].dot(item_vecs.T) 
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0] 
    recommend_vector = pref_vec*rec_vector_scaled 
    product_idx = np.argsort(recommend_vector)[::-1][:num_items] 
    rec_list = []
    for index in product_idx:
        code = item_list[index]
        rec_list.append([code]) 
    codes = [item[0] for item in rec_list]
    return codes

In [7]:
data = pd.read_csv('DATA.csv', usecols=[8,11])

In [8]:
data['rate'] = [1. for i in range(data.shape[0])]

In [37]:
le = LabelEncoder()
item_lookup = le.fit_transform(data[['mcc']])

In [24]:
customers = list(np.sort(data.customer_id.unique())) 
mccs = list(np.sort(data.mcc.unique()))
rates = list(data.rate)

data['customer_id'] = data['customer_id'].astype("category")
data['mcc'] = data['mcc'].astype("category")

UI_sparse = sparse.coo_matrix((data['rate'],
                   (data['mcc'].cat.codes.copy(),
                    data['customer_id'].cat.codes.copy()))).tocsr()

In [25]:
train, valid, users_altered = make_split(UI_sparse, validation_percent = 0.1)

In [26]:
max_ = 0
alp, reg = 0,0
# searching optimal hyperparametrs and then stopping by interrupt
while True:
    a = random.randint(1,20)
    j = random.randint(0,3)
    f = random.randint(5,30)
    it = random.randint(15,60)
    alpha = a
    user_vecs, item_vecs = implicit.alternating_least_squares(train*alpha, factors=f, 
                                                        regularization = 0.1/10**j, 
                                                        iterations = it)
    auc,_ = calc_mean_auc(train, users_altered, 
                  [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], valid)
    if auc>max_:
        max_,alp,reg,fac,iters = auc,a,j,f,it
        print(max_,alp,reg,fac,iters)

100%|████████████████████████████████████████████████████████████████████████████████| 50.0/50 [00:01<00:00, 40.33it/s]


0.69 5 0 16 50


100%|████████████████████████████████████████████████████████████████████████████████| 48.0/48 [00:01<00:00, 39.32it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 55.0/55 [00:01<00:00, 38.54it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 20.0/20 [00:00<00:00, 35.18it/s]


0.711 1 2 13 20


100%|████████████████████████████████████████████████████████████████████████████████| 32.0/32 [00:00<00:00, 39.47it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 17.0/17 [00:00<00:00, 39.30it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 53.0/53 [00:01<00:00, 35.50it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 27.0/27 [00:00<00:00, 41.02it/s]


...

100%|████████████████████████████████████████████████████████████████████████████████| 52.0/52 [00:01<00:00, 33.59it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 29.0/29 [00:00<00:00, 31.03it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 32.0/32 [00:00<00:00, 32.51it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 37.0/37 [00:01<00:00, 36.91it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 50.0/50 [00:01<00:00, 35.58it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 31.0/31 [00:01<00:00, 29.69it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 36.0/36 [00:01<00:00, 34.51it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 15.0/15 [00:00<00:00, 40.00it/s]
100%|███████████████████████████████████

100%|████████████████████████████████████████████████████████████████████████████████| 51.0/51 [00:01<00:00, 27.16it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 32.0/32 [00:00<00:00, 35.38it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 57.0/57 [00:01<00:00, 38.31it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 33.0/33 [00:00<00:00, 35.73it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 28.0/28 [00:00<00:00, 39.05it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 15.0/15 [00:00<00:00, 36.59it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 19.0/19 [00:00<00:00, 22.71it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 38.0/38 [00:01<00:00, 31.59it/s]
100%|███████████████████████████████████

100%|████████████████████████████████████████████████████████████████████████████████| 30.0/30 [00:00<00:00, 35.60it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 52.0/52 [00:01<00:00, 36.90it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 17.0/17 [00:00<00:00, 36.27it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 46.0/46 [00:01<00:00, 36.63it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 20.0/20 [00:00<00:00, 39.32it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 21.0/21 [00:00<00:00, 34.75it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 25.0/25 [00:00<00:00, 36.81it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 22.0/22 [00:00<00:00, 29.02it/s]
100%|███████████████████████████████████

100%|████████████████████████████████████████████████████████████████████████████████| 60.0/60 [00:01<00:00, 30.40it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 58.0/58 [00:02<00:00, 21.10it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 33.0/33 [00:01<00:00, 33.48it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 30.0/30 [00:00<00:00, 34.36it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 24.0/24 [00:00<00:00, 35.39it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 30.0/30 [00:00<00:00, 32.45it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 42.0/42 [00:01<00:00, 31.45it/s]
 74%|██████████████████████████████████████████████████████████▉                     | 40.5/55 [00:01<00:00, 36.52it/s]


KeyboardInterrupt: 

In [27]:
print(max_,alp,reg,fac,iters)

0.741 3 1 7 53


In [28]:
alpha,regularization,factors,iterations = 3, 0.01, 7, 53
model = implicit.als.AlternatingLeastSquares(factors=factors, 
                                                        regularization=regularization, 
                                                        iterations=iterations)
model.fit(UI_sparse2*alpha)

100%|████████████████████████████████████████████████████████████████████████████████| 53.0/53 [00:01<00:00, 36.93it/s]


In [34]:
customers_arr = np.array(customers)
mccs_arr = np.array(mccs)
user_d = user_dict(customers)

In [35]:
submission_als = pd.DataFrame()
i = 0
for cust in customers_arr:
    scores = model.recommend(user_d[cust], UI_sparse2.T, N=5)
    pred = [s[0] for s in scores]
    p = le.inverse_transform(pred)
    submission_als[cust]= p

In [36]:
submission_als

,0001f322716470bf9bfc1708f06f00fc,0007297d86e14bd68bd87b1dbdefe302,000b709c6c6fb1e8efcfd95e57c2a9de,0027a7618d97cc9fbda55fac457eaeb7,002b9f6e118c54f1292e03d1a04d516e,0031915eb230f772681fb5dc5a8d1c31,003360bff9882ca4a4f93394dd984822,0037f3de3d890df1022cc760a1dfd9d6,003fa58414cc55531fcc38423bea8f8e,0046c2952fb808aa11f74abce5abe097,...,ff70c3c5f2dca00f255705305d75111d,ff71bdbcba59047f1fad88dcb7052151,ff7a887d347a8d598dc8e559d3aaec2f,ff7e1c6c07469b28a07847540385e767,ff869ee855dc3f9b382c943eb43cc4ec,ff92d5420f5fb92a37e1280d1fc9e5f4,ffaeae55d4dbf29058f04e7a6a764f02,ffc5289194413ec68c3f7adc8121d69b,ffd097949a4a238296a7deadfb376cc0,ffdd5ec2a90e355cf40525eac1a6fd34
0,6011,4111,5261,6011,5691,5912,5411,6011,5912,5691,...,5977,6011,6011,6011,5411,6011,5411,6011,5261,5977
1,5411,5812,5541,5411,6011,5814,6011,5411,5812,5411,...,5691,5411,5411,5411,6011,5411,6011,5411,5541,5411
2,5814,5814,6011,5814,5411,5411,5331,5541,5814,6011,...,5411,5814,5814,5814,5499,5814,5814,5814,6011,6011
3,5499,5499,5411,5912,5912,6011,5499,5814,5411,5912,...,6011,5499,5499,4111,5814,5541,5912,5912,5411,5912
4,5912,5411,5814,5499,5977,5812,5211,5499,6011,5814,...,5912,5912,5912,5912,5331,5912,5499,5499,5814,5814


# Submit

In [38]:
submission_als.to_csv("submission_als_MF.csv", index=False)

In [295]:
submission_MF_warp.to_csv("submission_MF_warp.csv", index=False)

In [126]:
submission_NeuMF.to_csv("submission_NeuMF.csv", index=False)